## Transfer Learning using CIFAR-10 data
You will work with the CIFAR-10 Dataset. This is a well-known dataset for image classification, which consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The 10 classes are: 
airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

To illustrate the power and concept of transfer learning, you will train a CNN on just the classes (airplane, automobile, bird, cat, deer). Then you will train just the last layer(s) of the network on the classes (dog, frog, horse, ship, truck) and see how well the features learned on (airplane, automobile, bird, cat, deer) help with classifying (dog, frog, horse, ship, truck).


In [1]:
import datetime
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import RMSprop

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
## This just handles some variability in how the input data is loaded
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 3)

In [4]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=64, # I defined the values inside the function directly
              epochs=10,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [5]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

# create two datasets: one with classes below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

## Assignment
### PART-1: Build your CNN model
PART-1: Build your own model
Build a CNN model with the following specifications:

1. Two convolutional layers with ReLU activations.
2. MaxPooling with stride 2. Dropout of 0.25 after MaxPooling.
3. Two hidden fully-connected layers, two dropouts of 0.25 and a final output layer for classification.
4. Train this model for 10 epochs with RMSProp at a learning rate of .001 and a batch size of 64.
5. Evaluate the test results.


In [6]:
# hyperparamaters
epochs = 10
batch_size = 64
learning_rate = 0.001
num_classes = 5

In [7]:
# Define your model here
# Creating model with sequantial to add several layers
model = Sequential()
# First convolutional layer 
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))

# Second convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))

# MaxPooling layer with stride 2
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))

# Flatten 
model.add(Flatten())

# Adding first fully connected layer
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Adding second fully connected layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

# final output for classification with softmax activation for multi-class classification with 5 classes with num_classes=5
model.add(Dense(num_classes))
model.add(Activation('softmax'))

/Users/beyzakaya/miniforge3/envs/keras_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         1,285 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,337,573 (16.55 MB)

 Trainable params: 4,337,573 (16.55 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Now, train your model on the classes (airplane, automobile, bird, cat, deer)
train_model(model, (x_train_lt5, y_train_lt5), (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (25000, 32, 32, 3)
25000 train samples
5000 test samples
Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 77ms/step - accuracy: 0.4462 - loss: 1.3375 - val_accuracy: 0.6710 - val_loss: 0.8384
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 79ms/step - accuracy: 0.6692 - loss: 0.8381 - val_accuracy: 0.7114 - val_loss: 0.7256
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 85ms/step - accuracy: 0.7401 - loss: 0.6785 - val_accuracy: 0.7252 - val_loss: 0.7192
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 85ms/step - accuracy: 0.7942 - loss: 0.5565 - val_accuracy: 0.7454 - val_loss: 0.6915
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 96ms/step - accuracy: 0.8268 - loss: 0.4683 - val_accuracy: 0.7614 - val_loss: 0.6565
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - accuracy: 0.8604 - loss: 0.3814 - val_accuracy: 0.7654 - val_loss: 0.7082
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 94ms/step - accuracy: 0.8920 - loss: 0.2943 - val_accuracy: 0.7854 - val_loss: 0.6641
Epoch 8/10
391/391

### Evaluate the test results.
Test score corresponds to categorical cross entropy loss on the test dataset. Lower value of loss indicates better model but this value depends on the scale of the problem and complexity of the dataset. We can look for training and validation loss in each epoch to observe it is decrease or increase to understand model learning each step. Loss values both in training and validation decreases so model weights are updated in a way. We can look for accuracy values of validation to see model performance on test dataset. Test accuracy is 77% approximately which means model correctly predicts approximately %77 of the test dataset so accuracy metric measures the model performance on test dataset in terms of classification accuracy.

### PART-2: Transfer Learning
### Freezing Layers
Keras allows layers to be "frozen" during the training process. That is, some layers would have their weights updated during the training process, while others would not. This is a core part of transfer learning, the ability to train just the last one or several layers.

A model layer (at position i) can be frozen as follows:

model.layers[i].trainable = False

Now, fine-tune your model in two different ways and compare the performances:

1. Freeze all layers except the output layer, and train your model on the classes (dog, frog, horse, ship, truck).
2. Freeze all layers except the fully connected layer and the output layer, and train your model on the classes (dog, frog, horse, ship, truck). 

Compare the classification accuracy and training time of these two fine-tuning approaches and answer the following:

1. How many trainable parameters are there in each case?
2. Which fine-tuning performs better in terms of classification accuracy and why?
3. Why is fine-tuning much faster than the initial training of the network?


In [12]:
# Freeze all layers except the last one 
for layer in model.layers[:-2]:
    layer.trainable = False
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

Layer 0: conv2d, Trainable: False
Layer 1: activation, Trainable: False
Layer 2: conv2d_1, Trainable: False
Layer 3: activation_1, Trainable: False
Layer 4: max_pooling2d, Trainable: False
Layer 5: dropout, Trainable: False
Layer 6: flatten, Trainable: False
Layer 7: dense, Trainable: False
Layer 8: activation_2, Trainable: False
Layer 9: dropout_1, Trainable: False
Layer 10: dense_1, Trainable: False
Layer 11: activation_3, Trainable: False
Layer 12: dropout_2, Trainable: False
Layer 13: dense_2, Trainable: True
Layer 14: activation_4, Trainable: True


Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         1,285 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,675,148 (33.09 MB)

 Trainable params: 1,285 (5.02 KB)

 Non-trainable params: 4,336,288 (16.54 MB)

 Optimizer params: 4,337,575 (16.55 MB)

In [14]:
# Now, fine-tune your model on the classes (dog, frog, horse, ship, truck)
train_model(model, (x_train_gte5, y_train_gte5),(x_test_gte5, y_test_gte5), num_classes)

x_train shape: (25000, 32, 32, 3)
25000 train samples
5000 test samples
Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.2136 - loss: 4.4765 - val_accuracy: 0.5974 - val_loss: 1.0110
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.5789 - loss: 1.0502 - val_accuracy: 0.6446 - val_loss: 0.9463
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.6078 - loss: 0.9963 - val_accuracy: 0.6574 - val_loss: 0.9147
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.6256 - loss: 0.9694 - val_accuracy: 0.6668 - val_loss: 0.8958
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - accuracy: 0.6293 - loss: 0.9441 - val_accuracy: 0.6662 - val_loss: 0.8847
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.6282 - loss: 0.9554 - val_accuracy: 0.6720 - val_loss: 0.8736
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 47ms/step - accuracy: 0.6377 - loss: 0.9477 - val_accuracy: 0.6630 - val_loss: 0.8756
Epoch 8/10
391/391

In [15]:
# Freeze all layers first
for layer in model.layers:
    layer.trainable = False
# Trainable fully connected layers and the output layer between 7 and 14th layers
for layer in model.layers[7:14]:
    layer.trainable = True
# Printing the results for control
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

Layer 0: conv2d, Trainable: False
Layer 1: activation, Trainable: False
Layer 2: conv2d_1, Trainable: False
Layer 3: activation_1, Trainable: False
Layer 4: max_pooling2d, Trainable: False
Layer 5: dropout, Trainable: False
Layer 6: flatten, Trainable: False
Layer 7: dense, Trainable: True
Layer 8: activation_2, Trainable: True
Layer 9: dropout_1, Trainable: True
Layer 10: dense_1, Trainable: True
Layer 11: activation_3, Trainable: True
Layer 12: dropout_2, Trainable: True
Layer 13: dense_2, Trainable: True
Layer 14: activation_4, Trainable: False


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         1,285 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,665,004 (33.05 MB)

 Trainable params: 4,327,429 (16.51 MB)

 Non-trainable params: 10,144 (39.62 KB)

 Optimizer params: 4,327,431 (16.51 MB)

In [16]:
train_model(model, (x_train_gte5, y_train_gte5),(x_test_gte5, y_test_gte5), num_classes)

x_train shape: (25000, 32, 32, 3)
25000 train samples
5000 test samples
Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.7388 - loss: 0.7112 - val_accuracy: 0.8466 - val_loss: 0.4240
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.8669 - loss: 0.3805 - val_accuracy: 0.8682 - val_loss: 0.3766
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 50ms/step - accuracy: 0.9072 - loss: 0.2626 - val_accuracy: 0.8686 - val_loss: 0.3893
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 22s 57ms/step - accuracy: 0.9367 - loss: 0.1809 - val_accuracy: 0.8702 - val_loss: 0.3958
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 0.9599 - loss: 0.1200 - val_accuracy: 0.8742 - val_loss: 0.4223
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.9711 - loss: 0.0885 - val_accuracy: 0.8694 - val_loss: 0.5026
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.9751 - loss: 0.0767 - val_accuracy: 0.8782 - val_loss: 0.5019
Epoch 8/10
391/391

### 1. How many trainable parameters are there in each case?
In the original model there are 4,337,573 parameters in total and all the parameters are trainable so there are 4,337,573 trainable parameters. In the first version of fine-tuning, there are 1,285 trainable parameters because we froze the layers except output layer. In the second version of fine-tuning, there are 4,327,429 trainable parameters because we froze the layers except fully connected layers and the output layer. 

### 2. Which fine-tuning performs better in terms of classification accuracy and why?
Second version of the fine-tuning performs better in terms of classification accuracy because there are more trainable parameters in the second version. In the first version, we only use the output layer as a trainable which is very small part of the model so the second version beats the accuracy because trainable parameters are much higher than the first version because we train fully connected layers extra for first version.

### 3. Why is fine-tuning much faster than the initial training of the network?
Fine tuning froze desired layers and train only the small part of the architecture. Fine tuning has small number amount trainable parameters compare to the original training because original training update weights of the all architecture. Fine-tuning enhances the time by reducing the trainable number of parameters so this enbales the transfer learning by training a base model than fine-tuning on another task to predict on different dataset.